In [1]:
import time
import pandas as pd
import datetime
import os


In [2]:
# 讀取檔案資訊
datasite = './TimeSeriesData/TS5-201902_202306.parquet'
df = pd.read_parquet(datasite)

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./TimeSeriesData'
DataName = 'TS6-201902_202306'

df

,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,7DaysMA
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.846510,170.285714
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.846510,170.285714
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.846510,170.285714
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.846510,170.285714
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.846510,170.285714
...,...,...,...,...,...,...,...,...,...,...,...,...
30273907,yellow,2023,6,30,19,199,4,False,0,40.793489,-73.880683,0.142857
30273908,yellow,2023,6,30,20,199,4,False,0,40.793489,-73.880683,0.142857
30273909,yellow,2023,6,30,21,199,4,False,0,40.793489,-73.880683,0.142857
30273910,yellow,2023,6,30,22,199,4,False,0,40.793489,-73.880683,0.142857


In [3]:
# 記錄開始時間
start_time = time.time()

# 臨時日期
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str)+' '+df['hour'].astype(str),format='%Y-%m-%d %H')
#df['TempTime-7'] =  df['TempTime'] - pd.Timedelta(days=7)
#df['TempTime+7'] =  df['TempTime'] + pd.Timedelta(days=7)
df_minus7d = df.copy()
df_minus7d['TempTime'] = df_minus7d['TempTime'] - pd.Timedelta(days=7)
df_add7d = df.copy()
df_add7d['TempTime'] =  df_add7d['TempTime'] + pd.Timedelta(days=7)

# 創一個同車種、往前7天、同時段的數值 minus7d
merged_df = df.merge(df_add7d[['Name', 'TempTime','PULocationID','count']], on=['Name', 'TempTime','PULocationID'], how='left')
df['minus7d'] = merged_df['count_y']
# 創一個同車種、往後7天、同時段的數值 add7d
merged_df = df.merge(df_minus7d[['Name', 'TempTime','PULocationID','count']], on=['Name', 'TempTime','PULocationID'], how='left')
df['add7d'] = merged_df['count_y']

# 取(minus7d、count、add7d)平均 countN
df['countN']=(df['count']+df['add7d']+df['minus7d'])/3

# 移除col
df = df.drop(['minus7d'],axis = 1)
df = df.drop(['add7d'],axis = 1)
df = df.drop(['7DaysMA'],axis = 1)
df = df.drop(['TempTime'],axis = 1)
df['countN'].fillna(df['count'], inplace=True)

# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')
    

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time

print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())
df

save!
程式執行花費的時間： 125.13 秒
done時間: 2023-09-05 10:08:05.228412


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,countN
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.846510,4.0
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.846510,2.0
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.846510,6.0
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.846510,1.0
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.846510,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30273907,yellow,2023,6,30,19,199,4,False,0,40.793489,-73.880683,0.0
30273908,yellow,2023,6,30,20,199,4,False,0,40.793489,-73.880683,0.0
30273909,yellow,2023,6,30,21,199,4,False,0,40.793489,-73.880683,0.0
30273910,yellow,2023,6,30,22,199,4,False,0,40.793489,-73.880683,0.0
